In [43]:
import numpy as np
import pandas as pd
import xarray as xr
from itertools import permutations
import pyproj
from typing import Union
from osgeo import gdal, ogr, osr
from pathlib import Path
from pypism.extract_profile import normal, tangential

In [28]:
    def F(x, y, z):
        """A function linear in x, y, and z. Used to test our interpolation
        scheme."""
        return 10.0 + 0.01 * x + 0.02 * y + 0.03 + 0.04 * z



In [29]:
def create_dummy_input_dataset(F) -> xr.Dataset:
    """Create an input file for testing. Does not use unlimited
    dimensions, creates one time record only."""

    Mx = 88
    My = 152
    Mz = 11
    Mt = 1

    # use X and Y ranges corresponding to a grid covering Greenland
    x = np.linspace(-669650.0, 896350.0, Mx)
    y = np.linspace(-3362600.0, -644600.0, My)
    z = np.linspace(0, 4000.0, Mz)

    xx, yy = np.meshgrid(x, y)

    def write(dimensions: list):
        "Write test data to the file using given storage order."

        slices: dict[str, Any] = {
            "x": slice(0, Mx),
            "y": slice(0, My),
            "time": 0,
            "z": None,
        }
        dim_map = {"x": Mx, "y": My, "z": Mz, "time": Mt}

        # set indexes for all dimensions (z index will be re-set below)
        indexes: list[Any] = [Ellipsis] * len(dimensions)
        for k, d in enumerate(dimensions):
            indexes[k] = slices[d]

        # transpose 2D array if needed
        if dimensions.index("y") < dimensions.index("x"):

            def T(x):
                return x

        else:
            T = np.transpose

        dims = [dim_map[d] for d in dimensions]
        variable = np.zeros(dims)
        if "z" in dimensions:
            for k in range(Mz):
                indexes[dimensions.index("z")] = k
                variable[*indexes] = T(F(xx, yy, z[k]))
        else:
            variable[*indexes] = T(F(xx, yy, 0))

        return (dimensions, variable, {"long_name": name + " (make it long!)"})

    def P(x):
        return list(permutations(x))

    data_vars = {}
    for d in sorted(P(["x", "y"]) + P(["time", "x", "y"])):
        prefix = "test_2D_"
        name = prefix + "_".join(d)
        data_vars[name] = write(d)

    for d in sorted(P(["x", "y", "z"]) + P(["time", "x", "y", "z"])):
        prefix = "test_3D_"
        name = prefix + "_".join(d)
        data_vars[name] = write(d)

    ds = xr.Dataset(
        data_vars=data_vars,
        coords={
            "time": (["time"], [0], {}),
            "z": (["z"], z, {"_FillValue": False, "units": "m"}),
            "y": (
                ["y"],
                y,
                {
                    "_FillValue": False,
                    "units": "m",
                    "axis": "Y",
                    "standard_name": "projection_y_coordinate",
                },
            ),
            "x": (
                ["x"],
                x,
                {
                    "_FillValue": False,
                    "units": "m",
                    "axis": "X",
                    "standard_name": "projection_x_coordinate",
                },
            ),
        },
        attrs={"description": "Test data.", "proj": "epsg:3413", "proj4": "epsg:3413"},
    )
    return ds


In [30]:
dummy_input_dataset = create_dummy_input_dataset(F)

In [34]:
    "Create a dummy profile for testing."

    x = dummy_input_dataset["x"]
    y = dummy_input_dataset["y"]
    proj = dummy_input_dataset.proj
    projection = pyproj.Proj(str(proj))

    n_points = 4
    # move points slightly to make sure we can interpolate
    epsilon = 0.1
    x_profile = np.linspace(x[0] + epsilon, x[-1] - epsilon, n_points)
    y_profile = np.linspace(y[0] + epsilon, y[-1] - epsilon, n_points)
    x_center = 0.5 * (x_profile[0] + x_profile[-1])
    y_center = 0.5 * (y_profile[0] + y_profile[-1])

    lon, lat = projection(x_profile, y_profile, inverse=True)
    clon, clat = projection(x_center, y_center, inverse=True)

    flightline = 2
    glaciertype = 4
    flowtype = 2

    p = Profile(
        0,
        "test profile",
        lat,
        lon,
        clat,
        clon,
        flightline,
        glaciertype,
        flowtype,
        projection,
    )


In [35]:
p.c

In [21]:
p.center_lat

71.62733345812714

In [61]:
class Profile():

    """Collects information about a profile, that is a sequence of points
    along a flux gate or a flightline.

    """

    def __init__(
        self,
        id: int,
        name: str,
        lat: Union[float, np.ndarray],
        lon: Union[float, np.ndarray],
        center_lat: float,
        center_lon: float,
        flightline: int,
        glaciertype: int,
        flowtype: int,
        projection: str,
        flip: bool = False,
    ):
        self.id = id
        self.name = name
        self.center_lat = center_lat
        self.center_lon = center_lon
        self.flightline = flightline
        self.glaciertype = glaciertype
        self.flowtype = flowtype

        try:
            lon[0]
        except:
            lon = [lon]

        try:
            lat[0]
        except:
            lat = [lat]

        assert len(lon) == len(lat)

        if flip:
            self.lat = lat[::-1]
            self.lon = lon[::-1]
        else:
            self.lat = lat
            self.lon = lon
        self.x, self.y = projection(lon, lat)

        self.distance_from_start = self._distance_from_start()
        self.nx, self.ny = self._compute_normals()
        self.tx, self.ty = self._compute_tangentials()

    def _compute_normals(self):
        """
        Compute normals to a flux gate described by 'p'. Normals point 'to
        the right' of the path.
        """

        p = np.vstack((self.x, self.y)).T

        if len(p) < 2:
            return [0], [0]

        ns = np.zeros_like(p)
        ns[0] = normal(p[0], p[1])
        for j in range(1, len(p) - 1):
            ns[j] = normal(p[j - 1], p[j + 1])

        ns[-1] = normal(p[-2], p[-1])

        return ns[:, 0], ns[:, 1]

    def _compute_tangentials(self):
        """
        Compute tangetials to a flux gate described by 'p'.
        """

        p = np.vstack((self.x, self.y)).T

        if len(p) < 2:
            return [0], [0]

        ts = np.zeros_like(p)
        ts[0] = tangential(p[0], p[1])
        for j in range(1, len(p) - 1):
            ts[j] = tangential(p[j - 1], p[j + 1])

        ts[-1] = tangential(p[-2], p[-1])

        return ts[:, 0], ts[:, 1]

    def _distance_from_start(self):
        "Initialize the distance along a profile."
        result = np.zeros_like(self.x)
        result[1::] = np.sqrt(np.diff(self.x) ** 2 + np.diff(self.y) ** 2)
        return result.cumsum()


In [53]:

f = read_shapefile(Path("../tests/data/greenland-flux-gates-29_500m.gpkg"))

Spatial Reference System in ../tests/data/greenland-flux-gates-29_500m.gpkg is not latlon. Converting.


In [59]:
def test_read_shapefile():

    filenames = [Path("../tests/data/greenland-flux-gates-29_500m.shp"), Path("../tests/data/greenland-flux-gates-29_500m.gpkg")]

    for filename in filenames:

        profiles = read_shapefile(filename)
        
        assert len(profiles) == 28


In [60]:
test_read_shapefile()

Spatial Reference System in ../tests/data/greenland-flux-gates-29_500m.shp is not latlon. Converting.
Spatial Reference System in ../tests/data/greenland-flux-gates-29_500m.gpkg is not latlon. Converting.


In [54]:
len(f)

28

In [56]:
f[0]

[[66.33191641435869,
  66.33620492209785,
  66.34049346387123,
  66.34478203964998,
  66.34907064940536,
  66.35335929310848,
  66.3576479707305,
  66.36193668224257,
  66.3662254276158,
  66.37051420682134,
  66.37480301983022,
  66.37909186661358,
  66.38338074714245,
  66.3876696613879,
  66.391958609321,
  66.39624759091271,
  66.40053660613408],
 [-38.27358599884209,
  -38.27149223089385,
  -38.26939768273915,
  -38.26730235394739,
  -38.2652062440877,
  -38.26310935272887,
  -38.261011679439385,
  -38.258913223787424,
  -38.256813985340834,
  -38.25471396366718,
  -38.25261315833366,
  -38.25051156890722,
  -38.24840919495445,
  -38.24630603604163,
  -38.24420209173473,
  -38.242097361599406,
  -38.239991845201004],
 20,
 'Helheimgletscher',
 66.05382,
 -32.297,
 2,
 0,
 0]